# Tests on YoloV8 for fAIr
Using model V2 from Omdena results.
Mocking `test_yolo_v2.py`

In [1]:
# Standard library imports
import os
import time
import warnings
import ultralytics

os.environ.update(os.environ)
os.environ["RAMP_HOME"] = os.getcwd()

In [ ]:
# Reader imports
from hot_fair_utilities import polygonize, predict, preprocess
from hot_fair_utilities.preprocessing.yolo_v8_v2.yolo_format_anna import yolo_format
from hot_fair_utilities.training.yolo_v8_v2.train import train as train_yolo

warnings.simplefilter(action="ignore", category=FutureWarning)


Ramp eval metrics are not available, Possibly ramp is not installed


In [ ]:
# base_path = f"{os.getcwd()}/ramp-data/sample_2"
base_path = "/Users/azanchetta/fAIr_metric/training_results"
cities_list = "model51_td364"
# CHANGE PATH ACCORDING TO DATA STRUCTURE

# CALL FUNCTION FOR SELECTING THE FILES ACCORDING TO RAMP INPUT (will be in `test_fn.py`)

# Looping through the cities list, and check per each folder that tiles number is consistent, and the shapes too
for city in cities_list:
  city_folder_name = f'{base_path}/{city}'
  print(f'\n---\nNow working on {city}\n---')


class print_time:
    def __init__(self, name):
        self.name = name

    def __enter__(self):
        self.start = time.perf_counter()
        return self

    def __exit__(self, type, value, traceback):
        print(f"{self.name} took {round(time.perf_counter() - self.start, 2)} seconds")


start_time = time.time()

### SKIP fAIr processing, we already have the images ready
# model_input_image_path = f"{base_path}/input"
# preprocess_output = f"{base_path}/Yolo/preprocessed"
# with print_time("preprocessing"):
#     preprocess(
#         input_path=model_input_image_path,
#         output_path=preprocess_output,
#         rasterize=True,
#         rasterize_options=["binary"],
#         georeference_images=True,
#         multimasks=False,
#         epsg=4326
#     )

yolo_data_dir = f"{base_path}/yolo_v2"

with print_time("yolo conversion"):
    yolo_format(
        input_path=preprocess_output,
        output_path=yolo_data_dir,
    )


Clipping labels for input: 100%|██████████| 147/147 [00:11<00:00, 12.49it/s]

preprocessing took 15.71 seconds


In [ ]:
# Training

output_model_path,output_model_iou_accuracy = train_yolo(
    data=f"{base_path}",
    weights=f"{os.getcwd()}/yolov8s_v2-seg.pt", 
    # gpu="cpu",
    epochs=2,
    batch_size=16,
    pc=2.0,
    output_path=yolo_data_dir,
    dataset_yaml_path=os.path.join(yolo_data_dir,'yolo_dataset.yaml')
)
print(output_model_iou_accuracy)


In [ ]:
# Prediction
prediction_output = f"{base_path}/prediction/output"
# model_path = f"{output_path}/weights/best.pt"
with print_time("inference"):
    predict(
        checkpoint_path=output_model_path,
        input_path=f"{base_path}/prediction/input",
        prediction_path=prediction_output,
    )